In [1]:
# Dependencies
import pandas as pd


In [ ]:
# Save path to data set in a variable
#Food_data_file = pd.read_csv("data/FoodData.csv")


In [ ]:
# Use Pandas to read data
Food_data_file_df = pd.read_csv("data/FoodData.csv")
Food_data_file_df.head()

,Class,Type,Group,Food,Allergy
0,Plant origin,Nut and seed,Oil seed,Almond,Nut Allergy
1,Plant origin,Fruit,Pome fruit,Apple,Oral Allergy Syndrome
2,Plant origin,Fruit,Stone fruit,Apricot,Stone Fruit Allergy
3,Plant origin,Vegetable,Composite vegetable,Artichoke,Insulin Allergy
4,Plant origin,Vegetable,Liliaceous vegetable,Asparagus,Allium Allergy


In [2]:

Food_data_file_df = pd.read_csv("data/FoodData.csv")
Food_data_file_df.head(10)

,Class,Type,Group,Food,Allergy
0,Plant origin,Nut and seed,Oil seed,Almond,Nut Allergy
1,Plant origin,Fruit,Pome fruit,Apple,Oral Allergy Syndrome
2,Plant origin,Fruit,Stone fruit,Apricot,Stone Fruit Allergy
3,Plant origin,Vegetable,Composite vegetable,Artichoke,Insulin Allergy
4,Plant origin,Vegetable,Liliaceous vegetable,Asparagus,Allium Allergy
5,Plant origin,Fruit,Tropical fruit,Avocado,Oral Allergy Syndrome
6,Plant origin,Vegetable,Miscellaneous vegetable,Bamboo shoot,Histamine Allergy
7,Plant origin,Fruit,Tropical fruit,Banana,Banana Allergy
8,Plant origin,Cereal grain and pulse,Cereal grain,Barley,Gluten Allergy
9,Plant origin,Cereal grain and pulse,Pulse,Bean,Legume Allergy


In [3]:
# The unique method shows every element only once
unique = Food_data_file_df["Type"].unique()
unique

array(['Nut and seed', 'Fruit', 'Vegetable', 'Cereal grain and pulse',
       'Aquatic animal', 'Dairy', 'Oil', 'Terrestrial mammal', 'Poultry',
       'Bee product', 'Hop', 'Mineral water', 'Spice and herb', 'Tea'],
      dtype=object)

In [ ]:
count = Food_data_file_df["Type"].value_counts()
count

Vegetable                 64
Fruit                     34
Cereal grain and pulse    21
Aquatic animal            14
Nut and seed              12
Dairy                     11
Oil                       10
Terrestrial mammal         6
Poultry                    4
Spice and herb             3
Bee product                2
Hop                        1
Mineral water              1
Tea                        1
Name: Type, dtype: int64

In [ ]:
unique = Food_data_file_df["Food"].unique()
unique

array(['Almond', 'Apple', 'Apricot', 'Artichoke', 'Asparagus', 'Avocado',
       'Bamboo shoot', 'Banana', 'Barley', 'Bean', 'Blackberry',
       'Black-eyed bean', 'Blueberry', 'Bonito', 'Broad bean', 'Broccoli',
       'Brussels sprouts', 'Buckwheat', 'Burdock', 'Butter',
       'Butter bean', 'Buttermilk', 'Button mushroom', 'Cabbage',
       'Cacao bean', 'Canola oil', 'Carrot', 'Casein', 'Cattle',
       'Cauliflower', 'Celery', 'Cheese', 'Cherry', 'Chestnut', 'Chicken',
       'Chicory', 'Chinese cabbage', 'Coconut oil', 'Coffee bean', 'Corn',
       'Corn oil', 'Cotton seed', 'Cranberry', 'Cream', 'Crustaceans',
       'Cucumber ', 'Custard', 'Date', 'Deer', 'Duck', 'Eel', 'Egg plant',
       'Eggs', 'Endive', 'Fructose', 'Garlic', 'Ghrkin', 'Ginger',
       'Ginkgo nut', 'Globfish', 'Glucose', 'Goat', 'Grape', 'Grapefruit',
       'Grapeseed oil', 'Green soybean', 'Groundnut oil', 'Guava',
       'Honey', 'Hop', 'Horse', 'Horse Mackerel', 'Horseradish',
       'Huckleberry', 'I

In [ ]:
count = Food_data_file_df["Food"].value_counts()
count

Peas             2
Almond           1
Pegia            1
Peppermint       1
Percifomes       1
                ..
Grapefruit       1
Grapeseed oil    1
Green soybean    1
Groundnut oil    1
Yogurt           1
Name: Food, Length: 183, dtype: int64

In [ ]:
unique = Food_data_file_df["Group"].unique()
unique

array(['Oil seed', 'Pome fruit', 'Stone fruit', 'Composite vegetable',
       'Liliaceous vegetable', 'Tropical fruit',
       'Miscellaneous vegetable', 'Cereal grain', 'Pulse', 'Berry',
       'Fish', 'Cruciferous vegetable', 'Dairy', 'Mushroom',
       'Beverage seed', 'Oil', 'Umbelliferous vegetable', 'Offal',
       'Shellfish', 'Cucurbitaceous vegetable', 'Solanceous vegetable',
       'Egg', 'Sugar source', 'Grape', 'Citrus fruit / citric acid',
       'Legumes vegetable', 'Bee product', 'Hop', 'Potato',
       'Mineral water', 'Herb', 'Spice', 'Tea'], dtype=object)

In [ ]:

count = Food_data_file_df["Group"].value_counts()
count

Cruciferous vegetable         15
Pulse                         13
Fish                          12
Dairy                         11
Oil                           10
Oil seed                      10
Tropical fruit                10
Offal                          9
Cereal grain                   8
Liliaceous vegetable           7
Composite vegetable            7
Stone fruit                    7
Cucurbitaceous vegetable       7
Berry                          6
Sugar source                   6
Potato                         5
Citrus fruit / citric acid     5
Umbelliferous vegetable        5
Pome fruit                     5
Miscellaneous vegetable        4
Solanceous vegetable           3
Legumes vegetable              3
Shellfish                      2
Beverage seed                  2
Mushroom                       2
Bee product                    2
Herb                           2
Egg                            1
Grape                          1
Hop                            1
Mineral wa

In [ ]:
unique = Food_data_file_df["Class"].unique()
unique

array(['Plant origin', 'Animal origin', 'Mineral waters'], dtype=object)

In [ ]:
count = Food_data_file_df["Class"].value_counts()
count

Plant origin      146
Animal origin      37
Mineral waters      1
Name: Class, dtype: int64

In [ ]:
unique = Food_data_file_df["Allergy"].unique()
unique

array(['Nut Allergy', 'Oral Allergy Syndrome', 'Stone Fruit Allergy',
       'Insulin Allergy', 'Allium Allergy', 'Histamine Allergy',
       'Banana Allergy', 'Gluten Allergy', 'Legume Allergy',
       'Salicylate Allergy', 'Broccoli allergy', 'Cruciferous Allergy',
       'Ragweed Allergy', 'Milk allergy / Lactose intolerance',
       'Mushroom Allergy', nan, 'Hypersensitivity', 'Alpha-gal Syndrome',
       'Poultry Allergy', 'Ochratoxin Allergy', 'Corn Allergy',
       'Seed Allergy', 'Shellfish Allergy', 'Fish Allergy',
       'Nightshade Allergy', 'Sugar Allergy / Intolerance', 'LTP Allergy',
       'Citrus Allergy', 'Honey Allergy', 'Beer Allergy',
       'Potato Allergy', 'Lactose Intolerance', 'Aquagenic Urticaria',
       'Peanut Allergy', 'Mint Allergy', 'Rice Allergy', 'Pepper Allergy',
       'Soy Allergy', 'Tannin Allergy', 'Thyroid'], dtype=object)

In [ ]:
count = Food_data_file_df["Allergy"].value_counts()
count

Legume Allergy                        14
Oral Allergy Syndrome                 13
Fish Allergy                          12
Milk allergy / Lactose intolerance    11
Cruciferous Allergy                    9
Stone Fruit Allergy                    7
Allium Allergy                         7
Alpha-gal Syndrome                     6
Hypersensitivity                       6
Salicylate Allergy                     6
Sugar Allergy / Intolerance            5
Nut Allergy                            5
Seed Allergy                           5
Potato Allergy                         5
Histamine Allergy                      5
Insulin Allergy                        5
Citrus Allergy                         5
Gluten Allergy                         4
Poultry Allergy                        4
Nightshade Allergy                     3
Corn Allergy                           3
Mushroom Allergy                       2
LTP Allergy                            2
Honey Allergy                          2
Mint Allergy    